In [1]:
import os
import torch
import torch.nn as nn


from mmengine.config import Config
from mmselfsup.registry import DATA_SAMPLERS, DATASETS, MODELS, TRANSFORMS
from mmselfsup.models.utils import Extractor

from mmengine.dist import get_rank, init_dist
from mmengine.registry import init_default_scope

from mmengine.runner import Runner, load_checkpoint
from torch.utils.data import DataLoader

from mmengine.dataset import pseudo_collate, worker_init_fn, default_collate

cfg_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj.py'
ckpt_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/epoch_100.pth'

cfg = Config.fromfile(cfg_pth)

init_default_scope(cfg.get('default_scope', 'mmselfsup'))

/mindhive/nklab5/users/amarvi/anaconda3/envs/mmselfsup/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj.py'
ckpt_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/epoch_100.pth'

cfg = Config.fromfile(cfg_pth)

cfg.model
cfg.model.backbone.out_indices = [0,1,2,3,4,5,6,7,8,9,10]

model = MODELS.build(cfg.model)
model.init_weights()
load_checkpoint(model, ckpt_pth, map_location='cpu')

model = model.cuda()

08/04 13:39:48 - mmengine - WARNING - The value of momentum in EMA is usually a small number,which is different from the conventional notion of momentum but got 0.99. Please make sure the value is correct.
08/04 13:39:49 - mmengine - INFO - 
backbone.cls_token - torch.Size([1, 1, 768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 13:39:49 - mmengine - INFO - 
backbone.pos_embed - torch.Size([1, 197, 768]): 
Initialized by user-defined `init_weights` in VisionTransformer  
 
08/04 13:39:49 - mmengine - INFO - 
backbone.patch_embed.projection.weight - torch.Size([768, 3, 16, 16]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 13:39:49 - mmengine - INFO - 
backbone.patch_embed.projection.bias - torch.Size([768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 13:39:49 - mmengine - INFO - 
backbone.layers.0.ln1.weight - torch.Size([768]): 
The value is the same before and after calling

In [3]:
dataset_type = 'mmcls.CustomDataset'
data_root = '/om2/user/amarvi/FACE/data/behav_test/'
extract_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='mmcls.ResizeEdge', scale=256, edge='short'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackSelfSupInputs')
]

train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        pipeline=extract_pipeline),
    collate_fn=dict(type='default_collate'),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

# pooling cfg
# this isn't the right pool for ViT
# output of backbone is Tuple[List[Tensor]]]: (11, (2, [batch-size ,768, 14, 14]))
# 11 layers, 2 ???, batch-size, output of backbone, 14x14 patches?

# each layer outputs 2 tensors: [batch-size, 768, 14, 14] (image tokens) and [batch-size, 768] (clss token)
pool_cfg = dict(type='AvgPool2d_DINO')
# pool_cfg='test'

data_loader_train = Runner.build_dataloader(dataloader=train_dataloader)

extractor_train = Extractor(
        extract_dataloader=data_loader_train,
        dist_mode=False,
        pool_cfg=pool_cfg)

In [4]:
outputs = extractor_train(model)

[                                                  ] 0/200, elapsed: 0s, ETA:11 torch.Size([1, 768, 14, 14])
0 torch.Size([1, 768])
1 torch.Size([1, 768])
2 torch.Size([1, 768])
3 torch.Size([1, 768])
4 torch.Size([1, 768])
5 torch.Size([1, 768])
6 torch.Size([1, 768])
7 torch.Size([1, 768])
8 torch.Size([1, 768])
9 torch.Size([1, 768])
10 torch.Size([1, 768])
[                                 ] 1/200, 0.3 task/s, elapsed: 3s, ETA:   648s11 torch.Size([1, 768, 14, 14])
0 torch.Size([1, 768])
1 torch.Size([1, 768])
2 torch.Size([1, 768])
3 torch.Size([1, 768])
4 torch.Size([1, 768])
5 torch.Size([1, 768])
6 torch.Size([1, 768])
7 torch.Size([1, 768])
8 torch.Size([1, 768])
9 torch.Size([1, 768])
10 torch.Size([1, 768])
[                                 ] 2/200, 0.6 task/s, elapsed: 3s, ETA:   324s11 torch.Size([1, 768, 14, 14])
0 torch.Size([1, 768])
1 torch.Size([1, 768])
2 torch.Size([1, 768])
3 torch.Size([1, 768])
4 torch.Size([1, 768])
5 torch.Size([1, 768])
6 torch.Size([1, 768])


In [21]:
data = tuple([torch.randn(11, 768, 14, 14), torch.randn(768)] for _ in range(11))
image_tokens = tuple(layer_data[0] for layer_data in data)
image_tokens[0].squeeze().shape

avgpool = m = nn.AdaptiveAvgPool2d((1, 1))

outs = []
for x in image_tokens:
    x_ = avgpool(x)
    outs.append(x_)

flat_features = [feat.view(feat.size(0), -1) for feat in outs]
flat_features[0].shape

torch.Size([11, 768])

In [ ]:
init_default_scope(cfg.get('default_scope', 'mmselfsup'))
if cfg.env_cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '0'
os.environ['MASTER_ADDR'] = "127.0.0.1"
os.environ['MASTER_PORT'] = '29500'
os.environ['PORT'] = '29500'

init_dist('pytorch')